In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import glob
import pickle

In [13]:
teams = ['BKN', 'MIL', 'GSW', 'LAL', 'IND', 'CHA', 'CHI', 'DET',
          'WAS', 'TOR', 'BOS', 'NYK', 'CLE', 'MEM', 'PHI', 'NOP',
          'HOU','MIN', 'ORL', 'SAS', 'OKC', 'UTA', 'SAC', 'POR',
          'DEN', 'PHX', 'DAL', 'ATL', 'MIA', 'LAC']
window = 3 # time window in minutes.

In [3]:
# Find "active" players in each team
# active_players = dict()
# for team in teams:
#     df = pd.read_csv('./boxscores/boxscores.csv')
#     df['GAME_ID'] = df['GAME_ID'].astype(str)
#     df_team = df[(df['TEAM_ABBREVIATION'] == team) & (df['GAME_ID'].str[1:3] == '21')] # team games
#     num_of_games = len(df_team['GAME_ID'].unique())

#     players = sorted(df_team['PLAYER_NAME'].unique())
#     active = []
#     for p in players:
#         if len(df_team[df_team['PLAYER_NAME'] == p]['MIN'].notnull()) > 0.7 * num_of_games:
#             active.append(p)
#     active_players[team] = active

In [4]:
active_players = {team: list() for team in teams}
active_players['BKN'] = ['Spencer Dinwiddie', 'Cam Thomas', 'Mikal Bridges', 'Dorian Finney-Smith', 'Nic Claxton']
active_players['MIL'] = ['Jrue Holiday', 'Grayson Allen', 'Khris Middleton', 'Giannis Antetokounmpo', 'Bobby Portis']
active_players['GSW'] = ['Stephen Curry', 'Jordan Poole', 'Andrew Wiggins', 'Otto Porter Jr.', 'Kevon Looney']
active_players['LAL'] = ['Russell Westbrook', 'Malik Monk', 'LeBron James', 'Carmelo Anthony', 'Anthony Davis']
active_players['IND'] = ['Malcolm Brogdon', 'Chris Duarte', 'Oshae Brissett', 'Domantas Sabonis', 'Myles Turner']
active_players['CHA'] = ['LaMelo Ball', 'Terry Rozier', 'Kelly Oubre Jr.', 'Miles Bridges', 'Mason Plumlee']
active_players['CHI'] = ['Coby White', 'Zach LaVine', 'Javonte Green', 'DeMar DeRozan', 'Nikola Vucevic']
active_players['DET'] = ['Killian Hayes', 'Cade Cunningham', 'Saddiq Bey', 'Jerami Grant', 'Isaiah Stewart']
active_players['WAS'] = ['Raul Neto', 'Kentavious Caldwell-Pope', 'Deni Avdija', 'Kyle Kuzma', 'Daniel Gafford']
active_players['TOR'] = ['Fred VanVleet', 'Gary Trent Jr.', 'OG Anunoby', 'Scottie Barnes', 'Precious Achiuwa']
active_players['BOS'] = ['Marcus Smart', 'Jaylen Brown', 'Jayson Tatum', 'Grant Williams', 'Al Hordford']
active_players['NYK'] = ['Immanuel Quickley', 'Evan Fournier', 'RJ Barrett', 'Julius Randle', 'Mitchell Robinson']
active_players['CLE'] = ['Darius Garland', 'Lauri Markanen', 'Isaac Okoro', 'Evab Mobley', 'Jarrett Allen']
active_players['MEM'] = ['Ja Morant', 'Desmond Bane', 'Ziaire Williams', 'Jaren Jackson Jr.', 'Steven Adams']
active_players['PHI'] = ['Tyrese Maxey', 'Matisse Thybulle', 'Danny Green', 'Tobias Harris', 'Joel Embiid']
active_players['NOP'] = ["Devonte' Graham", 'Josh Hart', 'Brandon Ingram', 'Herbert Jones', 'Jonas Valanciunas']
active_players['HOU'] = ['Kevin Porter Jr.', 'Jalen Green', 'KJ Martin', "Jae'Sean Tate", 'Christian Wood']
active_players['MIN'] = ["D'Angleo Russell", 'Anthony Edwards', 'Leandro Bolmaro', 'Jarred Vanderbilt', 'Karl-Anthony Towns']
active_players['ORL'] = ['Cole Anthony', 'Gary Harris', 'Franz Wagner', 'Wendell Carter Jr.', 'Mo Bamba']
active_players['SAS'] = ['Dejounte Murray', 'Lonnie Walker IV', 'Keldon Johnson', 'Doug McDermott', 'Jakob Poeltl']
active_players['OKC'] = ['Shai Gilgeous-Alexander', 'Luguentz Dort', 'Josh Giddey', 'Darius Bazley', 'Jeremiah Robinson-Earl']
active_players['UTA'] = ['Mike Conley', 'Donovan Mitchell', "Royce O'Neale", 'Bojan Bogdanovic', 'Rudy Gobert']
active_players['SAC'] = ["De'Aaron Fox'", 'Tyresse Haliburton', 'Maurice Harkless', 'Harrison Barnes', 'Chimezie Metu']
active_players['POR'] = ['Damian Lillard', 'Anfernee Simons', 'Normal Powell', 'Robert Covington', 'Jusuf Nurkic']
active_players['DEN'] = ['Monte Morris', 'Will Barton', 'Jeff Green', 'Aaron Gordon', 'Nikola Jokic']
active_players['PHX'] = ['Chris Paul', 'Devin Booker', 'Mikal Bridges', 'Jae Crowder', 'Deandre Ayton']
active_players['DAL'] = ['Luka Doncic', 'Jalen Brunson', 'Reggie Bullock', 'Dorian Finney-Smith', 'Dwight Powell']
active_players['ATL'] = ['Trae Young', 'Bogdan Bogdanovic', 'Kevin Huerter', 'Danilo Gallinari', 'Clint Capela']
active_players['MIA'] = ['Kyle Lowry', 'Tyler Herro', 'Jimmy Butler', 'P.J. Tucker', 'Bam Adebayo']
active_players['LAC'] = ['Eric Bledsoe', 'Reggie Jackson', 'Terance Mann', 'Marcus Morris', 'Ivica Zubac']
with open('./starters.pickle', 'wb') as file:
    pickle.dump(active_players, file)

In [5]:
files = glob.glob("./2021-2022_NBA_PbP_Logs/*")
files = sorted(files)

In [6]:
# df = pd.read_csv('[2021-10-20]-0022100005-BOS@NYK.csv')
# df.columns

In [15]:
def calcPlayerStats(team, df):
    players = active_players[team]
    player_stats = {p : [] for p in players}
    start = 0
    period = 1

    while period <= 4:
        end_str = str(start + window).zfill(2)
        start_str = str(start).zfill(2)

        df_window = df[(df['period'] == period) & (df['elapsed'] >= f'0:{start_str}:00') & (df['elapsed'] <= f'0:{end_str}:00')]

        # Calculate PIE for each player
        #  PIE = (PTS + FGM + FTM – FGA – FTA + Deff.REB + Off.REB/2 + AST + STL + BLK/2 – PF – TO) / 
        #        (GAME_PTS + GAME_FGM + ... - GAME_PF - GAME_TO)
        # Calculate Game stats
        G_PTS = df_window['points'].sum(skipna=True)        
        G_FGM = len(df_window[(df_window['event_type'] == 'shot') & (df_window['result'] == 'made')])
        G_FGA = len(df_window[(df_window['event_type'] == 'shot')])
        G_FTM = len(df_window[(df_window['event_type'] == 'free throw') & (df_window['result'] == 'made')])
        G_FTA = len(df_window[(df_window['event_type'] == 'free throw')])
        G_AST = len(df_window[df_window['assist'].notnull()])

        G_DEF_REB = len(df_window[df_window['type'] == 'rebound defensive'])
        G_OFF_REB = len(df_window[df_window['type'] == 'rebound offensive'])

        G_STL = len(df_window[df_window['steal'].notnull()])
        G_BLK = len(df_window[df_window['block'].notnull()])
        G_PF = len(df_window[df_window['event_type'] == 'foul'])
        G_TO = len(df_window[df_window['event_type'] == 'turnover'])

        game_sum = G_PTS + G_FGM + G_FTM - G_FGA - G_FTA + G_DEF_REB + G_OFF_REB / 2 + G_AST + G_STL + G_BLK/2 - G_PF - G_TO
            
        for p in players:
            df_player = df_window[(df_window['a1'] == p) | (df_window['a2'] == p) | (df_window['a3'] == p) 
                                  | (df_window['a4'] == p) | (df_window['a5'] == p) | (df_window['h1'] == p)
                                  | (df_window['h2'] == p) | (df_window['h3'] == p) | (df_window['h4'] == p)
                                  | (df_window['h5'] == p)]
            
            # Calculate PIE for each player
            PTS = df_player[(df_player['event_type'] == 'shot') & (df_player['player'] == p)]['points'].sum(skipna=True)

            FGM = len(df_player[(df_player['event_type'] == 'shot') 
                                & (df_player['result'] == 'made') 
                                & df_player['player'] == p])
            FGA = len(df_player[(df_player['event_type'] == 'shot') & df_player['player'] == p])
            FTM = len(df_player[(df_player['event_type'] == 'free throw') 
                                & (df_player['result'] == 'made')
                                & (df_player['player'] == p)])
            FTA = len(df_player[(df_player['event_type'] == 'free throw') & df_player['player'] == p])
            AST = len(df_player[df_player['assist'] == p])

            DEF_REB = len(df_player[(df_player['player'] == p) & (df_player['type'] == 'rebound defensive')])
            OFF_REB = len(df_player[(df_player['type'] == 'rebound offensive') & (df_player['player'] == p)])

            STL = len(df_player[df_player['steal'] == p])
            BLK = len(df_player[df_player['block'] == p])
            PF = len(df_player[(df_player['player'] == p) & (df_player['event_type'] == 'foul')])
            TO = len(df_player[(df_player['player'] == p) & (df_player['event_type'] == 'turnover')])
            
            player_sum = PTS + FGM + FTM - FGA - FTA + DEF_REB + OFF_REB / 2 + AST + STL + BLK/2 - PF - TO
            
            # total time in seconds that player p played during the window
            df_play_length = df_player['play_length']
            time = df_play_length.agg(lambda col : sum(int(t[-2:]) for t in col)) if len(df_play_length) else 0
            
            
            if game_sum == 0:
                PIE = np.nan
            else:
                PIE = player_sum / game_sum
                
            # Add stats to dict
            player_stats[p].append((PIE, player_sum, game_sum, time))
        
        # update window
        start += window
        if start >= 12:
            start = 0 
            period += 1
        
    return player_stats

In [8]:
all_teams_stats = {team: {} for team in teams}
for team in teams:
    print(f'Calculating stats for {team} ...')
    files_team = [f for f in files if team in f]
    
    for f in files_team:
        df = pd.read_csv(f)
        game_id = f[-22:-12]
        player_stats = calcPlayerStats(team, df)

        all_teams_stats[team][game_id] = player_stats  


Calculating stats for BKN ...
Calculating stats for MIL ...
Calculating stats for GSW ...
Calculating stats for LAL ...
Calculating stats for IND ...
Calculating stats for CHA ...
Calculating stats for CHI ...
Calculating stats for DET ...
Calculating stats for WAS ...
Calculating stats for TOR ...
Calculating stats for BOS ...
Calculating stats for NYK ...
Calculating stats for CLE ...
Calculating stats for MEM ...
Calculating stats for PHI ...
Calculating stats for NOP ...
Calculating stats for HOU ...
Calculating stats for MIN ...
Calculating stats for ORL ...
Calculating stats for SAS ...
Calculating stats for OKC ...
Calculating stats for UTA ...
Calculating stats for SAC ...
Calculating stats for POR ...
Calculating stats for DEN ...
Calculating stats for PHX ...
Calculating stats for DAL ...
Calculating stats for ATL ...
Calculating stats for MIA ...
Calculating stats for LAC ...


In [9]:
with open('./all_teams_PBP_data.pickle', 'wb') as file:
    pickle.dump(all_teams_stats, file)

# Diving deeper into specific game

## 20 May 2022 DAL@GSW

In [17]:
df = pd.read_csv('./2021-2022_NBA_PbP_Logs/[2022-05-20]-0042100312-DAL@GSW.csv')
player_stats = calcPlayerStats('DAL', df)
player_stats

{'Luka Doncic': [(0.6153846153846154, 8.0, 13.0, 159),
  (0.5833333333333334, 7.0, 12.0, 200),
  (0.25806451612903225, 4.0, 15.5, 169),
  (0.45454545454545453, 5.0, 11.0, 192),
  (0.0, 0.0, 14.0, 0),
  (0.0, 0.0, 16.5, 10),
  (0.14634146341463414, 3.0, 20.5, 188),
  (0.25, 5.0, 20.0, 185),
  (0.21428571428571427, 1.5, 7.0, 167),
  (0.0, 0.0, 4.0, 192),
  (0.16666666666666666, 1.0, 6.0, 167),
  (0.32, 4.0, 12.5, 194),
  (0.0, 0.0, 11.0, 0),
  (0.09523809523809523, 2.0, 21.0, 104),
  (0.4878048780487805, 10.0, 20.5, 174),
  (0.15789473684210525, 3.0, 19.0, 190)],
 'Jalen Brunson': [(0.3076923076923077, 4.0, 13.0, 159),
  (0.16666666666666666, 2.0, 12.0, 174),
  (0.0, 0.0, 15.5, 0),
  (0.0, 0.0, 11.0, 0),
  (0.2857142857142857, 4.0, 14.0, 180),
  (0.24242424242424243, 4.0, 16.5, 187),
  (0.2926829268292683, 6.0, 20.5, 188),
  (0.1, 2.0, 20.0, 185),
  (0.7142857142857143, 5.0, 7.0, 167),
  (0.25, 1.0, 4.0, 192),
  (0.0, 0.0, 6.0, 51),
  (0.0, 0.0, 12.5, 0),
  (0.2727272727272727, 3.0, 11.0